# Experiment 0

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [30]:
# Get data
experiment = '0'

model_names = ['ngc0', 'ngc', 'nri']
result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in model_names]

results = {
    'models': [],
    'accs': [],
    'seeds': [],
    'trainset_size': [],
    'tvt_split': [],
}

for i, result_file_ in enumerate(result_files):
    with open(result_file_) as f:
        results_file = json.load(f)

        results['models'] += [
            result_exp['experiment_config']['model'] for result_exp in results_file.values()
        ]
        results['accs'] += [
            result_exp['model_results']['accuracy'] for result_exp in results_file.values()
        ]
        results['seeds'] += [
            result_exp['experiment_config']['seed'] for result_exp in results_file.values()
        ]
        results['trainset_size'] += [
            result_exp['experiment_config']['trainset_size'] for result_exp in results_file.values()
        ]
        results['tvt_split'] += [
            result_exp['experiment_config']['tvt_split'] for result_exp in results_file.values()
        ]

0.9 0.9 0.0 0.9


In [314]:
# Save data in csv
df_ = pd.DataFrame(data=results)
df_.to_csv(f'results/{experiment}/processed_results_{experiment}.csv')
df_

,models,accs,seeds,trainset_size,tvt_split
0,ngc0,1.0,1,500,0.1
1,ngc0,1.0,101,500,0.1
2,ngc0,1.0,33,500,0.1
3,ngc0,1.0,12,500,0.1
4,ngc0,1.0,0,500,0.1
...,...,...,...,...,...
445,nri,0.5,1,5000,1.0
446,nri,0.5,101,5000,1.0
447,nri,0.5,33,5000,1.0
448,nri,0.5,12,5000,1.0


In [320]:
# Load the data and aggregate results
df = pd.read_csv(f'results/{experiment}/processed_results_{experiment}.csv', index_col='Unnamed: 0')
#df = df.groupby(by=['trainset_size', 'tvt_split']).mean()
df

,models,accs,seeds,trainset_size,tvt_split
0,ngc0,1.0,1,500,0.1
1,ngc0,1.0,101,500,0.1
2,ngc0,1.0,33,500,0.1
3,ngc0,1.0,12,500,0.1
4,ngc0,1.0,0,500,0.1
...,...,...,...,...,...
445,nri,0.5,1,5000,1.0
446,nri,0.5,101,5000,1.0
447,nri,0.5,33,5000,1.0
448,nri,0.5,12,5000,1.0


In [322]:
# Plot data


# Experiment 1a / 1b / 1c / 1d

In [353]:
import pandas as pd
import json

In [480]:
# Get data
experiment = '1d'

model_names = ['ngc', 'nri', 'ngc0', 'tvar']
result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in model_names]

results = {
    'models': [],
    'accs': [],
    'seeds': [],
    'c11': [],
    'c12': [],
    'c21': [],
    'c22': []
}

experiments = {}
for i, result_file_ in enumerate(result_files):
    with open(result_file_) as f:
        results_file = json.load(f)

        results['models'] += [
            result_exp['experiment_config']['model'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]
        results['accs'] += [
            result_exp['model_results']['accuracy'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]
        results['seeds'] += [
            result_exp['experiment_config']['seed'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]
        results['c11'] += [
            result_exp['experiment_config']['c11'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]
        results['c12'] += [
            result_exp['experiment_config']['c12'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]
        results['c21'] += [
            result_exp['experiment_config']['c21'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]
        results['c22'] += [
            result_exp['experiment_config']['c22'] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']
        ]

In [481]:
df = pd.DataFrame(data=results)
df

,models,accs,seeds,c11,c12,c21,c22
0,ngc,1.00,1,0.4,0.0,0.0,0.4
1,ngc,1.00,101,0.4,0.0,0.0,0.4
2,ngc,0.50,33,0.4,0.0,0.0,0.4
3,ngc,0.75,12,0.4,0.0,0.0,0.4
4,ngc,0.50,0,0.4,0.0,0.0,0.4
...,...,...,...,...,...,...,...
115,tvar,1.00,1,0.4,0.5,0.5,0.4
116,tvar,1.00,101,0.4,0.5,0.5,0.4
117,tvar,1.00,33,0.4,0.5,0.5,0.4
118,tvar,1.00,12,0.4,0.5,0.5,0.4


In [482]:
df.to_csv(f'results/{experiment}/processed_results_{experiment}.csv')
df = pd.read_csv(f'results/{experiment}/processed_results_{experiment}.csv', index_col='Unnamed: 0')
df

,models,accs,seeds,c11,c12,c21,c22
0,ngc,1.00,1,0.4,0.0,0.0,0.4
1,ngc,1.00,101,0.4,0.0,0.0,0.4
2,ngc,0.50,33,0.4,0.0,0.0,0.4
3,ngc,0.75,12,0.4,0.0,0.0,0.4
4,ngc,0.50,0,0.4,0.0,0.0,0.4
...,...,...,...,...,...,...,...
115,tvar,1.00,1,0.4,0.5,0.5,0.4
116,tvar,1.00,101,0.4,0.5,0.5,0.4
117,tvar,1.00,33,0.4,0.5,0.5,0.4
118,tvar,1.00,12,0.4,0.5,0.5,0.4


In [483]:
# Plot data

# Experiment 2

In [21]:
import pandas as pd

In [22]:
# Get data
experiment = '2'

model_names = ['ngc', 'nri', 'ngc0', 'tvar']
result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in model_names]

results = {
    'models': [],
    'accs': [],
    'seeds': [],
    'c11': [],
    'c12': [],
    'c21': [],
    'c22': [],
    'sigma_eta_diag': [],
    'sigma_eta_off_diag': []
}

experiments = {}
for i, result_file_ in enumerate(result_files):
    with open(result_file_) as f:
        results_file = json.load(f)

        results['models'] += [
            result_exp['experiment_config']['model'] for result_exp in results_file.values()
        ]
        results['accs'] += [
            result_exp['model_results']['accuracy'] for result_exp in results_file.values()
        ]
        results['seeds'] += [
            result_exp['experiment_config']['seed'] for result_exp in results_file.values()
        ]
        results['c11'] += [
            result_exp['experiment_config']['c11'] for result_exp in results_file.values()
        ]
        results['c12'] += [
            result_exp['experiment_config']['c12'] for result_exp in results_file.values()
        ]
        results['c21'] += [
            result_exp['experiment_config']['c21'] for result_exp in results_file.values()
        ]
        results['c22'] += [
            result_exp['experiment_config']['c22'] for result_exp in results_file.values()
        ]
        results['sigma_eta_diag'] += [
            result_exp['experiment_config']['sigma_eta_diag'] for result_exp in results_file.values()
        ]
        results['sigma_eta_off_diag'] += [
            result_exp['experiment_config']['sigma_eta_off_diag'] for result_exp in results_file.values()
        ]

In [23]:
df = pd.DataFrame(data=results)
df

,models,accs,seeds,c11,c12,c21,c22,sigma_eta_diag,sigma_eta_off_diag
0,ngc,1.0,1,0.8,0.0,0.0,0.8,0.01,0.00
1,ngc,1.0,101,0.8,0.0,0.0,0.8,0.01,0.00
2,ngc,1.0,33,0.8,0.0,0.0,0.8,0.01,0.00
3,ngc,1.0,12,0.8,0.0,0.0,0.8,0.01,0.00
4,ngc,1.0,0,0.8,0.0,0.0,0.8,0.01,0.00
...,...,...,...,...,...,...,...,...,...
155,tvar,1.0,1,0.8,0.8,0.0,0.8,0.02,0.02
156,tvar,1.0,101,0.8,0.8,0.0,0.8,0.02,0.02
157,tvar,1.0,33,0.8,0.8,0.0,0.8,0.02,0.02
158,tvar,1.0,12,0.8,0.8,0.0,0.8,0.02,0.02


In [24]:
df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,models,accs,seeds,c11,c12,c21,c22,sigma_eta_diag,sigma_eta_off_diag
0,ngc,1.0,1,0.8,0.0,0.0,0.8,0.01,0.00
1,ngc,1.0,101,0.8,0.0,0.0,0.8,0.01,0.00
2,ngc,1.0,33,0.8,0.0,0.0,0.8,0.01,0.00
3,ngc,1.0,12,0.8,0.0,0.0,0.8,0.01,0.00
4,ngc,1.0,0,0.8,0.0,0.0,0.8,0.01,0.00
...,...,...,...,...,...,...,...,...,...
155,tvar,1.0,1,0.8,0.8,0.0,0.8,0.02,0.02
156,tvar,1.0,101,0.8,0.8,0.0,0.8,0.02,0.02
157,tvar,1.0,33,0.8,0.8,0.0,0.8,0.02,0.02
158,tvar,1.0,12,0.8,0.8,0.0,0.8,0.02,0.02


# Experiment 3

In [465]:
import pandas as pd

In [9]:
# Get data
experiment = '3'

model_names = ['nri']
result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in model_names]

results = {
    'models': [],
    'accs': [],
    'seeds': [],
    'c11': [],
    'c12': [],
    'c21': [],
    'c22': [],
    'n_data': [],
}

experiments = {}
for i, result_file_ in enumerate(result_files):
    with open(result_file_) as f:
        results_file = json.load(f)

        results['models'] += [
            result_exp['experiment_config']['model'] for result_exp in results_file.values()
        ]
        results['accs'] += [
            result_exp['model_results']['accuracy'] for result_exp in results_file.values()
        ]
        results['seeds'] += [
            result_exp['experiment_config']['seed'] for result_exp in results_file.values()
        ]
        results['c11'] += [
            result_exp['experiment_config']['c11'] for result_exp in results_file.values()
        ]
        results['c12'] += [
            result_exp['experiment_config']['c12'] for result_exp in results_file.values()
        ]
        results['c21'] += [
            result_exp['experiment_config']['c21'] for result_exp in results_file.values()
        ]
        results['c22'] += [
            result_exp['experiment_config']['c22'] for result_exp in results_file.values()
        ]
        results['n_data'] += [
            result_exp['experiment_config']['n_data'] for result_exp in results_file.values()
        ]

In [10]:
df = pd.DataFrame(data=results)
df

,models,accs,seeds,c11,c12,c21,c22,n_data
0,nri,0.500000,1,0.8,0.1,0.1,0.8,2
1,nri,0.000000,101,0.8,0.1,0.1,0.8,2
2,nri,0.500000,33,0.8,0.1,0.1,0.8,2
3,nri,0.500000,12,0.8,0.1,0.1,0.8,2
4,nri,0.500000,0,0.8,0.1,0.1,0.8,2
5,nri,0.555556,1,0.8,0.1,0.1,0.8,3
6,nri,0.555556,101,0.8,0.1,0.1,0.8,3
7,nri,0.666667,33,0.8,0.1,0.1,0.8,3
8,nri,0.555556,12,0.8,0.1,0.1,0.8,3
9,nri,0.555556,0,0.8,0.1,0.1,0.8,3


# Experiment 4

In [11]:
import pandas as pd

In [12]:
# Get data
experiment = '4'

model_names = ['ngc']
result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in model_names]

results = {
    'models': [],
    'accs': [],
    'seeds': [],
    'c11': [],
    'c12': [],
    'c21': [],
    'c22': [],
    'sparsity': [],
}

experiments = {}
for i, result_file_ in enumerate(result_files):
    with open(result_file_) as f:
        results_file = json.load(f)

        results['models'] += [
            result_exp['experiment_config']['model'] for result_exp in results_file.values()
        ]
        results['accs'] += [
            result_exp['model_results']['accuracy'] for result_exp in results_file.values()
        ]
        results['seeds'] += [
            result_exp['experiment_config']['seed'] for result_exp in results_file.values()
        ]
        results['c11'] += [
            result_exp['experiment_config']['c11'] for result_exp in results_file.values()
        ]
        results['c12'] += [
            result_exp['experiment_config']['c12'] for result_exp in results_file.values()
        ]
        results['c21'] += [
            result_exp['experiment_config']['c21'] for result_exp in results_file.values()
        ]
        results['c22'] += [
            result_exp['experiment_config']['c22'] for result_exp in results_file.values()
        ]
        results['sparsity'] += [
            result_exp['experiment_config']['sparsity'] for result_exp in results_file.values()
        ]

In [13]:
df = pd.DataFrame(data=results)
df

,models,accs,seeds,c11,c12,c21,c22,sparsity
0,ngc,1.0,1,0,0,0,0,0.1
1,ngc,1.0,101,0,0,0,0,0.1
2,ngc,1.0,33,0,0,0,0,0.1
3,ngc,1.0,12,0,0,0,0,0.1
4,ngc,1.0,0,0,0,0,0,0.1
5,ngc,1.0,1,0,0,0,0,0.2
6,ngc,1.0,101,0,0,0,0,0.2
7,ngc,1.0,33,0,0,0,0,0.2
8,ngc,1.0,12,0,0,0,0,0.2
9,ngc,1.0,0,0,0,0,0,0.2


# Experiment 5

In [3]:
import pandas as pd
import json

In [4]:
# Get data
experiment = '5'
experiments = ['5a', '5b', '5c', '5d']

model_names = ['ngc', 'ngc0', 'nri', 'tvar']
result_files = [f'results/{experiment}/{ex}/{model_name}/results.json' for model_name in model_names for ex in experiments]

results = {
    'models': [],
    'accs': [],
    'seeds': [],
    'c11': [],
    'c12': [],
    'c21': [],
    'c22': []
}

experiments = {}
for i, result_file_ in enumerate(result_files):
    with open(result_file_) as f:
        results_file = json.load(f)

        results['models'] += [
            result_exp['experiment_config']['model'] for result_exp in results_file.values()
        ]
        results['accs'] += [
            result_exp['model_results']['accuracy'] for result_exp in results_file.values()
        ]
        results['seeds'] += [
            result_exp['experiment_config']['seed'] for result_exp in results_file.values()
        ]
        results['c11'] += [
            result_exp['experiment_config']['c11'] for result_exp in results_file.values()
        ]
        results['c12'] += [
            result_exp['experiment_config']['c12'] for result_exp in results_file.values()
        ]
        results['c21'] += [
            result_exp['experiment_config']['c21'] for result_exp in results_file.values()
        ]
        results['c22'] += [
            result_exp['experiment_config']['c22'] for result_exp in results_file.values()
        ]

In [6]:
df = pd.DataFrame(data=results)
df.to_csv(f'results/5/processed_results_{experiment}.csv')
df

,models,accs,seeds,c11,c12,c21,c22
0,ngc,0.75,1,0.9,0.4,0.0,0.9
1,ngc,0.75,101,0.9,0.4,0.0,0.9
2,ngc,0.75,33,0.9,0.4,0.0,0.9
3,ngc,0.75,12,0.9,0.4,0.0,0.9
4,ngc,0.75,0,0.9,0.4,0.0,0.9
...,...,...,...,...,...,...,...
75,tvar,0.50,1,0.9,0.0,0.0,0.9
76,tvar,0.50,101,0.9,0.0,0.0,0.9
77,tvar,0.25,33,0.9,0.0,0.0,0.9
78,tvar,0.50,12,0.9,0.0,0.0,0.9


In [8]:
df = pd.read_csv(f'results/5/processed_results_{experiment}.csv', index_col='Unnamed: 0')
df

,models,accs,seeds,c11,c12,c21,c22
0,ngc,0.75,1,0.9,0.4,0.0,0.9
1,ngc,0.75,101,0.9,0.4,0.0,0.9
2,ngc,0.75,33,0.9,0.4,0.0,0.9
3,ngc,0.75,12,0.9,0.4,0.0,0.9
4,ngc,0.75,0,0.9,0.4,0.0,0.9
...,...,...,...,...,...,...,...
75,tvar,0.50,1,0.9,0.0,0.0,0.9
76,tvar,0.50,101,0.9,0.0,0.0,0.9
77,tvar,0.25,33,0.9,0.0,0.0,0.9
78,tvar,0.50,12,0.9,0.0,0.0,0.9
